In [2]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
from icecream import ic, install
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [3]:
conn = connect("../../django/TargetDB/db.sqlite3")
html_dir = "html"

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

name = "All Targets"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_ephemerides,
    tlc.add_speckle,
    tlc.add_pepsi,
    tlc.add_lists,
    partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
    partial(tlc.add_side_status, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)
html.render_observing_pages(tl, None, {}, "../../Observing Files/All Targets")